In [ ]:
import pyarrow.csv as pac
import numpy as np
import pandas as pd
import duckdb
from project_fds.location_cleaner import LocationCleaner
from collections import Counter
from scipy.stats import ks_2samp
import re
import gc
import ast
from dateutil import parser
from datetime import date
import time

# **1. ANIME TABLE CLEANING**

## Preprocessing, comparison, cleaning and merging of datasets: **azatoth42** (full version), **azatoth42** (cleaned version) & **hernan4444** (full version):

* we will call **azathoth42** dataset as ***original***, **azathoth42-cleaned** as ***main*** dataset, and **hernan4444** as ***secondary*** dataset

========================================================

Number of unique titles:

**original**: **14478**

**main**: **6668**

**secondary**: **17562**

========================================================

In [6]:
import os
os.chdir("/home/yaroslav/FCUL/MARS_1.0")

In [7]:
secondary_df = pd.read_csv('data/datasets/anime_hernan4444/anime.csv')
original_df = pd.read_csv('data/datasets/anime_azathoth42/AnimeList.csv')
main_df = pd.read_csv('data/datasets/anime_azathoth42/anime_cleaned.csv')

In [42]:
print(f"secondary dataset columns:\n----------------\n{secondary_df.columns}")
print(f"\noriginal dataset columns:\n----------------\n{original_df.columns}")
print(f"\nmain dataset columns:\n----------------\n{main_df.columns}")
print("\nNumber of unique anime:")
print(f"secondary: {len(set(secondary_df["MAL_ID"]))}")
print(f"original: {len(set(original_df["anime_id"]))}")
print(f"main: {len(set(main_df["anime_id"]))}")

secondary dataset columns:
----------------
Index(['MAL_ID', 'Name', 'Score', 'Genres', 'English name', 'Japanese name',
       'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity',
       'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'],
      dtype='object')

original dataset columns:
----------------
Index(['anime_id', 'title', 'title_english', 'title_japanese',
       'title_synonyms', 'image_url', 'type', 'source', 'episodes', 'status',
       'airing', 'aired_string', 'aired', 'duration', 'rating', 'score',
       'scored_by', 'rank', 'popularity', 'members', 'favorites', 'background',
       'premiered', 'broadcast', 'related', 'producer', 'licensor', 'studio',
       'genre', 'opening_theme', 'ending_theme'],
      dtype='object')

### Preprocessing
### Get only the columns we will work with

- We should remove columns related to ***secondary*** dataset which based in any sense on users or user votes, because we use the users and users votes tables only from ***main***. These are: score_X_secondary. Also we remove columns "background_main" and "broadcast_main".

There are more of them. But discus them later.

In [44]:
"""
Primarly in secondary:
==========================
'MAL_ID', 'Name', 'Score', 'Genres', 'English name', 'Japanese name',
       'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity',
       'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'
"""
secondary_df.drop(
    columns=['Japanese name','Producers', 'Licensors', 'Ranked', 'Popularity', 'Watching',
             'Completed', 'On-Hold', 'Dropped', 'Plan to Watch', 'Score-10', 'Score-9',
             'Score-8', 'Score-7', 'Score-6', 'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'],
             inplace=True)

In [45]:
"""
Primarly in main:
==========================
'anime_id', 'title', 'title_english', 'title_japanese',
       'title_synonyms', 'image_url', 'type', 'source', 'episodes', 'status',
       'airing', 'aired_string', 'aired', 'duration', 'rating', 'score',
       'scored_by', 'rank', 'popularity', 'members', 'favorites', 'background',
       'premiered', 'broadcast', 'related', 'producer', 'licensor', 'studio',
       'genre', 'opening_theme', 'ending_theme', 'duration_min',
       'aired_from_year'
"""
main_df.drop(
    columns=['title_japanese', 'status', 'airing', 'aired_string', 'duration', 'popularity',
             'background', 'broadcast', 'related', 'producer', 'licensor'],
             inplace=True)

### Metrics comparison
##### (**original** (2018) vs **secondary** (2020))

We see that p-value is very small due to big amount of titles. In this case it is not suitable metric.
KS-statistic is more suitable. It shows a mild drift to bigger scores within 2 years. But almost no drift in average members and average favorites.

In [46]:
original_metric = np.array(original_df[original_df['score'] != 'Unknown']['score'], dtype=float)
secondary_metric = np.array(secondary_df[secondary_df['Score'] != 'Unknown']['Score'], dtype=float)
print(f"{'Score'}:\t{ks_2samp(original_metric, secondary_metric)}")
print(f"{'Score'}:\t{np.mean(original_metric)}, {np.mean(secondary_metric)}")
print(f"{'Score'}:\t{np.std(original_metric)}, {np.std(secondary_metric)}\n")

for metric in ['Members', 'Favorites']:
    original_metric = np.array(original_df[original_df[metric.lower()] != 'Unknown'][metric.lower()], dtype=int) / len(original_df)
    secondary_metric = np.array(secondary_df[secondary_df[metric] != 'Unknown'][metric], dtype=int) / len(secondary_df)
    print(f"{metric}:\t{ks_2samp(original_metric, secondary_metric)}")
    print(f"{metric}:\t{np.mean(original_metric)}, {np.mean(secondary_metric)}")
    print(f"{metric}:\t{np.std(original_metric)}, {np.std(secondary_metric)}\n")

Score:	KstestResult(statistic=np.float64(0.11009226884152352), pvalue=np.float64(5.056203583857332e-71), statistic_location=np.float64(5.21), statistic_sign=np.int8(1))
Score:	6.142481696366902, 6.509999194911844
Score:	1.4639305139139216, 0.8866814542368036

Members:	KstestResult(statistic=np.float64(0.0423449633394031), pvalue=np.float64(8.397513523923764e-13), statistic_location=np.float64(0.010360547036883548), statistic_sign=np.int8(1))
Members:	1.586296634889361, 1.973496157450135
Members:	5.178806945428294, 7.133502706100809

Favorites:	KstestResult(statistic=np.float64(0.07978029457698221), pvalue=np.float64(2.2021609447723465e-44), statistic_location=np.float64(5.6941122878943175e-05), statistic_sign=np.int8(-1))
Favorites:	0.021525736033928204, 0.02606458662774464
Favorites:	0.18065091043085962, 0.23137214568274153



### What changes cleaning made?

We see a noticeable shift to bigger scores, average members and average favorites, due to the fact that cleaning primarly acted to the low-scored, almost unknown anime titles.

Also we see that after cleaning std of the score was reduced, because cleaned anime are mostly in the right tail, in "outlier region", where titles can have only a few votes.

In [47]:
original_metric = np.array(original_df[original_df['score'] != 'Unknown']['score'], dtype=float)
main_metric = np.array(main_df[main_df['score'] != 'Unknown']['score'], dtype=float)
print(f"{'score'}:\t{np.mean(original_metric)}, {np.mean(main_metric)}")
print(f"{'score'}:\t{np.std(original_metric)}, {np.std(main_metric)}\n")

for metric in ['members', 'favorites']:
    original_metric = np.array(original_df[original_df[metric] != 'Unknown'][metric], dtype=int) / len(original_metric)
    main_metric = np.array(main_df[main_df[metric] != 'Unknown'][metric], dtype=int) / len(main_metric)
    print(f"{metric}:\t{np.mean(original_metric)}, {np.mean(main_metric)}")
    print(f"{metric}:\t{np.std(original_metric)}, {np.std(main_metric)}\n")

score:	6.142481696366902, 6.848998200359928
score:	1.4639305139139216, 0.9273779813068229

members:	1.586296634889361, 7.122130980222672
members:	5.178806945428294, 15.763837029693017

favorites:	0.021525736033928204, 0.100534737083777
favorites:	0.18065091043085962, 0.5733032618744727



In [48]:
del original_df
gc.collect()

1201

========================================================

Number of titles from **main**, that are missed in **secondary**: **12**

========================================================

In [49]:
set_of_anime_id_secondary = set(secondary_df["MAL_ID"])
set_of_anime_id_main = set(main_df["anime_id"])
main_secondary_diff = set_of_anime_id_main - set_of_anime_id_secondary
print(f"Number of anime ID in main dataset missing for secondary: {len(main_secondary_diff)}")
print(f"\nExact ID's:\n{main_secondary_diff}")

Number of anime ID in main dataset missing for secondary: 12

Exact ID's:
{34848, 10945, 36519, 36327, 37032, 37005, 12723, 10520, 32089, 31930, 37019, 16287}


========================================================

What if we remove these titles from **main** dataset?

We see that the distribution is the same. Removing of these titles is close to the removing of 12 random titles.

========================================================

In [50]:
df_cut = main_df[~main_df['anime_id'].isin(main_secondary_diff)]
print(ks_2samp(main_df['score'], df_cut['score']))

KstestResult(statistic=np.float64(0.0008187965772230169), pvalue=np.float64(1.0), statistic_location=np.float64(6.84), statistic_sign=np.int8(1))


### So, we can use this cut and do a merge. Columns, which are presented in both datasets, we name with suffix *"_main"* or *"_secondary"*. We will merge them later, using the **secondary** dataset to fill missing values in the **main** dataset.

In [51]:
main_df = df_cut.copy()
main_df = main_df.rename(columns={'genre': 'genres', 'studio': 'studios'})

secondary_df = secondary_df.rename(columns={col: ("anime_id" if col == "MAL_ID" else f"{col.lower().replace(" ", "_").replace("-", "_")}") for col in secondary_df.columns})
main_df = main_df.rename(columns={col: ("anime_id" if col == "anime_id" else f"{col.lower().replace(" ", "_").replace("-", "_")}") for col in main_df.columns})

common_columns = set(secondary_df.columns).intersection(set(main_df.columns))
common_columns.remove('anime_id')

secondary_df = secondary_df.rename(columns={col: (col if col not in common_columns else f"{col}_secondary") for col in secondary_df.columns})
main_df = main_df.rename(columns={col: (col if col not in common_columns else f"{col}_main") for col in main_df.columns})
df_main = pd.merge(main_df, secondary_df, on="anime_id", how="left")
df_main = df_main.sort_values("anime_id").reset_index(drop=True)
df_main.columns

Index(['anime_id', 'title', 'title_english', 'title_synonyms', 'image_url',
       'type_main', 'source_main', 'episodes_main', 'aired_main',
       'rating_main', 'score_main', 'scored_by', 'rank', 'members_main',
       'favorites_main', 'premiered_main', 'studios_main', 'genres_main',
       'opening_theme', 'ending_theme', 'duration_min', 'aired_from_year',
       'name', 'score_secondary', 'genres_secondary', 'english_name',
       'type_secondary', 'episodes_secondary', 'aired_secondary',
       'premiered_secondary', 'studios_secondary', 'source_secondary',
       'duration', 'rating_secondary', 'members_secondary',
       'favorites_secondary'],
      dtype='object')

In [52]:
del secondary_df
gc.collect()

0

### SOURCE, TYPE, STUDIO columns cleaning

========================================================

Then we will merge the pairs "X_main" & "X_secondary". For example - "type", "source", etc.

- In the cases of conflicts we will prefer values from **secondary** dataset (if such present) as:

- - Blind check (for every column - 10 random manual checks in the MyAnimeList website) shows that in these cases the **secondary** dataset is much more often presents "the groung truth" ***in all columns***. By "ground truth" we understand real value on the website at the moment of blind check.

- - It is logical because the **secondary** dataset is parse 2 years later than the **original**.

- In the case of missing values in both datasets we...will see how many it will be.

========================================================

For the source values we see that the difference mostly related to the misty borders of the, for example, manga and web-manga. Such differences are so speculative. We believe in the **secondary** dataset.

In [53]:
print("Source counters: main and secondary:")
print(Counter(df_main["source_main"]))
print(Counter(df_main["source_secondary"]))

df_main.rename(columns={"source_secondary": "source"}, inplace=True)
df_main.drop(columns=["source_main"], inplace=True)

print("\nType counters: main and secondary:")
print(Counter(df_main["type_main"]))
print(Counter(df_main["type_secondary"]))

df_main.rename(columns={"type_secondary": "type"}, inplace=True)
df_main.drop(columns=["type_main"], inplace=True)

print("\nStudio counters: main and secondary:")
print(Counter(df_main["studios_main"]))
print(Counter(df_main["studios_secondary"]))

df_main.rename(columns={"studios_secondary": "studio"}, inplace=True)
df_main.drop(columns=["studios_main"], inplace=True)

print("\nAre there still 'Unknown' values present?")
print('Unknown' in df_main["studio"], 'Unknown' in df_main["source"], 'Unknown' in df_main["type"])

Source counters: main and secondary:
Counter({'Manga': 2490, 'Original': 1785, 'Light novel': 511, 'Visual novel': 481, 'Game': 424, 'Novel': 281, '4-koma manga': 194, 'Other': 169, 'Web manga': 111, 'Music': 55, 'Picture book': 53, 'Book': 45, 'Card game': 45, 'Digital manga': 7, 'Radio': 5})
Counter({'Manga': 2479, 'Original': 1785, 'Light novel': 522, 'Visual novel': 482, 'Game': 430, 'Novel': 268, '4-koma manga': 196, 'Other': 172, 'Web manga': 117, 'Picture book': 54, 'Music': 52, 'Book': 44, 'Card game': 43, 'Digital manga': 7, 'Radio': 5})

Type counters: main and secondary:
Counter({'TV': 2980, 'OVA': 1344, 'Special': 920, 'Movie': 906, 'ONA': 408, 'Music': 98})
Counter({'TV': 2976, 'OVA': 1344, 'Special': 914, 'Movie': 912, 'ONA': 412, 'Music': 98})

Studio counters: main and secondary:
Counter({'Toei Animation': 402, 'Sunrise': 277, 'Madhouse': 243, 'Studio Pierrot': 235, 'J.C.Staff': 233, 'Studio Deen': 204, 'Production I.G': 177, 'A-1 Pictures': 158, 'TMS Entertainment': 15

### NAME column cleaning

========================================================

Try to choose one of the 'name'-related columns as base. In fact we need these columns only to orient through data, so try to choose the most filled & interpreteble column.

In the result of some analysis below we see (and addition researching of sources proves), that columns 'title' and 'name' are the most consistent. They are both parsed from one field: main title from the webpage of the anime title. And they are mostly the same: difference - in 418 of 6656 titles. But this diference s only syntaxis or the rare characters encoding-decoding issues. In the **secondary** dataset these string values are postprocessed with more acuracy. 

========================================================

In [54]:
for col in ['title', 'title_english', 'title_synonyms', 'name', 'english_name']:
    print(col)
    print("NaN:", df_main[col].isna().sum())
    print(df_main[col].isin(['Unknown']).value_counts())
    print()

neq, tot = 0, 0
for _, row in df_main.iterrows():
    if row['title'] != row['name']:
        print(f"{row['title']} | {row['name']}")
        neq += 1
    tot += 1

print(f"{neq} | {tot}")

title
NaN: 0
title
False    6656
Name: count, dtype: int64

title_english
NaN: 3219
title_english
False    6656
Name: count, dtype: int64

title_synonyms
NaN: 2183
title_synonyms
False    6656
Name: count, dtype: int64

name
NaN: 0
name
False    6656
Name: count, dtype: int64

english_name
NaN: 0
english_name
False    3528
True     3128
Name: count, dtype: int64

Beet the Vandel Buster | Bouken Ou Beet
Ghost in the Shell | Koukaku Kidoutai
Black Cat | Black Cat (TV)
Mahou Shoujo Lyrical Nanoha A&#039;s | Mahou Shoujo Lyrical Nanoha A's
Mobile Suit Gundam: The 08th MS Team - Miller&#039;s Report | Mobile Suit Gundam: The 08th MS Team - Miller's Report
Mobile Suit Gundam: Char&#039;s Counterattack | Mobile Suit Gundam: Char's Counterattack
Kidou Shin Seiki Gundam X | After War Gundam X
Hunter x Hunter: Yorkshin City Kanketsu-hen | Hunter x Hunter: Original Video Animation
Wolf&#039;s Rain | Wolf's Rain
R.O.D OVA | R.O.D: Read or Die
R.O.D the TV | R.O.D: The TV
E&#039;s Otherwise | E's O

In [55]:
df_main.drop(columns=['title', 'title_english', 'title_synonyms', 'english_name'], inplace=True)

### GENRES column cleaning

To merge genres columns we make a union of genres sets of both columns: *genres_main* & *genres_secondary*. We see that two anime titles has 'Unknown' in both datasets. Remove them.

In [56]:
for col in ['genres_main', 'genres_secondary']:
    df_main[col] = df_main[col].fillna("").apply(lambda s: set(x.strip() for x in s.split(", ")) if s else set())

df_main['genres'] = df_main.apply(lambda r: r['genres_main'] | r['genres_secondary'], axis=1)
df_main = df_main[df_main['genres'] != {'Unknown'}]
df_main.drop(columns=['genres_main', 'genres_secondary'], inplace=True)

### RATING (age category) column cleaning

First let us look on columns

In [57]:
nan_only_main, nan_only_sec, nan_both, both_not_nan_but_diff, tot = 0, 0, 0, 0, 0
rating_set = set()
max_score = 0
min_rank = 999999

for _, row in df_main.iterrows():
    main_nan_cond = pd.isnull(row['rating_main'])
    sec_nan_cond = (row['rating_secondary'] == 'Unknown')

    nan_only_main += int(main_nan_cond and not sec_nan_cond)
    nan_only_sec += int(sec_nan_cond and not main_nan_cond)
    nan_both += int(main_nan_cond and sec_nan_cond)
    both_not_nan_but_diff += not (main_nan_cond or sec_nan_cond) and (row['rating_main'] != row['rating_secondary'])

    rating_set.add(row['rating_main'])
    rating_set.add(row['rating_secondary'])
    if main_nan_cond and sec_nan_cond:
        if row['score_main'] > max_score:
            max_score = row['score_main']
        if row['rank'] < min_rank:
            min_rank = row['rank']

    tot += 1

print(f"only_main_ds_NaNs:\t{nan_only_main}\nonly_sec_ds_NaNs:\t{nan_only_sec}\nboth_nans:\t{nan_both}\nboth_not_nan_but_diff:\t{both_not_nan_but_diff}\ntotal:\t{tot}")
print(f"\nCANDIATES TO REMOVE (both nans: {both_not_nan_but_diff}):\nmax_score:\t{max_score}\nmin_rank:\t{min_rank}")
print(f"\nRatings set:\t{rating_set}")

only_main_ds_NaNs:	35
only_sec_ds_NaNs:	0
both_nans:	46
both_not_nan_but_diff:	47
total:	6654

CANDIATES TO REMOVE (both nans: 47):
max_score:	7.42
min_rank:	1927.0

Ratings set:	{'G - All Ages', 'Rx - Hentai', 'R+ - Mild Nudity', 'PG-13 - Teens 13 or older', 'PG - Children', 'R - 17+ (violence & profanity)', nan, 'Unknown'}


We see that if we apropriate following strategy:

- if one of two ratings is missed, fill with the second one;

- if both are missed, remove this anime title;

- if both are koen but differ, than fill with more strong,

then we will remove 46 rows. The minimal rank (the highest position) is 1927, and maximal score is 7.42.
Obviously, these are not so culturaly significant titles. So we can do it.

In [58]:
order = {
    'G - All Ages': 0,
    'PG - Children': 1,
    'PG-13 - Teens 13 or older': 2,
    'R - 17+ (violence & profanity)': 3,
    'R+ - Mild Nudity': 4,
    'Rx - Hentai': 5
}

def is_missing(x):
    return pd.isna(x) or str(x).strip().lower() in ['unknown', 'nan', '']

def merge_rating(main, sec):
    if is_missing(main) and is_missing(sec):
        return np.nan
    if is_missing(main):
        return sec
    if is_missing(sec):
        return main
    return main if order[main] >= order[sec] else sec

print(len(df_main))
df_main['rating'] = df_main.apply(lambda r: merge_rating(r['rating_main'], r['rating_secondary']), axis=1)
df_main.drop(columns=['rating_main', 'rating_secondary'], inplace=True)
df_main = df_main[df_main['rating'].notna()]
print(len(df_main))

6654
6608


### MEMBERS, FAVORITE column cleaning

These columns are clean.

In [59]:
for col in ['members', 'favorites']:
    df_main[col] = df_main[f"{col}_main"]
    df_main.drop(columns=[f"{col}_main", f"{col}_secondary"], inplace=True)

### AIRING, EPISODES column cleaning

As airing we mean such columns: *'aired_main'*, *'premiered_main'*, *'aired_from_year'*, *'aired_secondary'*, *'premiered_secondary'*. In common they are so dirty.

The most full information contains in *aired_* columns.

*aired_main* column contains values such **{'from': '1998-04-01', 'to': '1998-09-30'}** (a string, not a dictionary!) where every value can be **None**.

*aired_secondary* column contains values such **Apr 3, 1998 to Apr 24, 1999** or for one-episode items: **May 25, 2018**. Can also contain *'Unknown'*.

We will prefer *aired_secondary* column in the cases of conflicts.

Then drop all other columns related to dates.

Column **episodes** is closely related to airing, because the number of episodes is changing with time.

In [60]:
def normalize_date(x):
    """any string -> YYYY-MM-DD."""
    if not isinstance(x, str):
        return None

    x = x.strip()

    # "2000"
    if len(x) == 4 and x.isdigit():
        return f"{x}-01-01"

    try:
        dt = parser.parse(x)
        return dt.strftime("%Y-%m-%d")
    except:
        return None

def parse_secondary(sec_str):
    if not isinstance(sec_str, str):
        return None, None

    parts = sec_str.split(' to ')
    start = parts[0].strip()

    if len(parts) == 1:
        return start, start

    end = parts[1].strip()
    return start, end


def parse_main(main_obj):
    if isinstance(main_obj, str) and main_obj.startswith('{'):
        main_obj = ast.literal_eval(main_obj)

    if isinstance(main_obj, dict):
        return main_obj.get('from'), main_obj.get('to')

    return None, None


def merge_aired(row):
    mf, mt = parse_main(row['aired_main'])
    sf, st = parse_secondary(row['aired_secondary'])

    start = sf if sf else mf
    end   = st if (st and st != '?') else mt

    if start and not end:
        end = None  # may be it is ongoing

    # normalize (to ISO)
    start = normalize_date(start)
    end   = normalize_date(end)

    return start, end


df_main[['air_start', 'air_end']] = df_main.apply(merge_aired, axis=1, result_type='expand')

df_main['year_start'] = pd.to_datetime(df_main['air_start']).dt.year.astype('Int64')
df_main['year_finish'] = pd.to_datetime(df_main['air_end']).dt.year.astype('Int64')

df_main['month_start'] = pd.to_datetime(df_main['air_start']).dt.month_name()
df_main['month_finish'] = pd.to_datetime(df_main['air_end']).dt.month_name()

df_main.drop(columns=['aired_main', 'premiered_main', 'aired_from_year', 'aired_secondary', 'premiered_secondary', 'air_start', 'air_end'], inplace=True)
df_main.columns

Index(['anime_id', 'image_url', 'episodes_main', 'score_main', 'scored_by',
       'rank', 'opening_theme', 'ending_theme', 'duration_min', 'name',
       'score_secondary', 'type', 'episodes_secondary', 'studio', 'source',
       'duration', 'genres', 'rating', 'members', 'favorites', 'year_start',
       'year_finish', 'month_start', 'month_finish'],
      dtype='object')

### EPISODES column cleaning

There are only 31 anime titles which have missed values both in **main** and **secondary** tables. Below we discuss how to fill them. Any strategy has it's own cons, but zeros will be worse.

In [61]:
df_main['episodes'] = np.where(
    df_main['episodes_main'] != 0,
    df_main['episodes_main'],
    np.where(
        df_main['episodes_secondary'] != 'Unknown',
        df_main['episodes_secondary'],
        np.nan
    )
)

df_main.drop(columns=['episodes_main', 'episodes_secondary'], inplace=True)

print("Need to fill through requests: ", (df_main['episodes'].isna().sum()))

Need to fill through requests:  31


We can notice one interesting thing:

In [62]:
bad_year_finish = set(df_main[df_main['year_finish'].isna()]['anime_id'])
bad_episodes = set(df_main[df_main['episodes'].isna()]['anime_id'])

print(f"bad years of finish: {len(bad_year_finish)}")
print(f"bad episod numbers: {len(bad_episodes)}")
print(f"crosssection: {len(bad_year_finish.intersection(bad_episodes))}")

bad years of finish: 33
bad episod numbers: 31
crosssection: 31


So, in most cases these are the same anime titles. It seems that they were ongoings at those moment of time.

Big part of them have rank bigger than 10_000. But not all. For example, here we also have items with ranks 91, 272, 1032. It's so high. We can not remove these items. So we can parse these values from MAL website. It is very inefficient to do it without suitable API, but we don't need a big amount of data.

But here is another problem.

- Some of these 31-33 anime titles really have *'Unknown'* in the **episodes** field on the website.

- Even if some titles has this field filled - is this title finished before 2018? In most of cases - no. Some of them are still airing. So field **episodes** will contain number of episodes for the moment of parsing - 2025.

We have two options:

1. To parse different anime fan-hub. But anyway - i there are no missing data there - there will be data for 2025.

2. Second way seems more balanced:

- 1. Find anime titles with mising **episodes** in both tables. Filter only those which was finished before 2018. Let it be set **P**.
- 2. Parse the number of episodes for them from MAL webste (by the way will try to fill date of airing finish, if it earlier then 2018).
- 3. Ask ChatGPT to fill missing episodes fields for ***all*** problem titles, ask it to find information about number of episodes in the 2018 (archives, forums, etc.).
- 4. Compare ChatGPT's results with parsed results for animes from the set **P**. If in the most cases they will (almost) the same, take ChtGPT's results.

- * by the way we can fill the date of finish airing if it is good and if it is before 2018.

In [63]:
import requests

def get_episodes_and_finish(anime_id: int, timeout: int = 10):
    url = f"https://myanimelist.net/anime/{anime_id}"
    headers = {"User-Agent": "Mozilla/5.0"}
    resp = requests.get(url, headers=headers)
    html = resp.text
    
    # look for <span class="dark_text">Episodes:</span> ... 
    match = re.search(r'<span class="dark_text">Episodes:</span>\s*([^<\n]+)', html)
    if match:
        value = match.group(1).strip()
        if value.isdigit():
            ep = int(value)
        else:
            ep = None

    m = re.search(r'<span class="dark_text">Aired:</span>\s*([^<\n]+)', html)
    if m:
        aired_str = m.group(1).strip()
    return ep, aired_str


def get_episodes_and_finish_from_list(anime_id_set: set, fake_run=True, verbose=False) -> dict:
    real_dict = {
        33922: [9, 'Apr 6, 2016 to Mar 27, 2023'], 6149: [None, 'Jan 8, 1995 to ?'],
        34566: [293, 'Apr 5, 2017 to Mar 26, 2023'], 35590: [11, 'May 5, 2017 to Mar 15, 2020'],
        22669: [None, 'Apr 25, 2011 to ?'], 8336: [None, 'Mar 29, 2010 to ?'],
        24977: [58, 'Jul 26, 2014 to Mar 6, 2023'], 21: [None, 'Oct 20, 1999 to ?'],
        35478: [10, 'May 6, 2017 to Mar 16, 2018'], 37020: [52, 'Oct 2, 2017 to Mar 22, 2019'],
        34845: [None, 'Mar 13, 2014 to ?'], 35229: [82, 'Nov 29, 2013 to Jun 7, 2018'],
        1960: [None, 'Oct 3, 1988 to ?'], 34092: [3, 'Sep 17, 2016 to Sep 25, 2017'],
        28205: [1, 'Oct 30, 2014'], 1199: [None, 'Apr 10, 1993 to ?'],
        29361: [1, 'Feb 6, 2019'], 36409: [6, 'Sep 22, 2017 to Aug 14, 2019'],
        32956: [230, 'Apr 8, 2016 to Mar 4, 2022'], 37187: [20, 'Jul 19, 2014 to Dec 27, 2014'],
        24773: [9, 'Oct 11, 2013 to Mar 7, 2020'], 966: [None, 'Apr 13, 1992 to ?'],
        24775: [None, 'Jan 3, 2015 to ?'], 24793: [26, 'Jul 18, 2012 to ?'],
        32353: [None, 'Apr 2, 2016 to ?'], 2406: [None, 'Oct 5, 1969 to ?'],
        34535: [21, 'Mar 4, 2017 to 2020'], 37096: [None, 'Apr 2, 2018 to ?'],
        235: [None, 'Jan 8, 1996 to ?'], 4459: [None, 'Oct 5, 1998 to ?'],
        8687: [None, 'Apr 22, 2005 to ?'], 23539: [1717, 'Mar 31, 2014 to Dec 24, 2023'], 18941: [None, 'Apr 2, 2012 to ?']}
    if fake_run:
        return real_dict

    dict_id_episodes = {}
    for aid in anime_id_set:
        if verbose: print(f"Read for id {aid}...")
        ep, aired = get_episodes_and_finish(aid)
        dict_id_episodes[aid] = [ep, aired]
        if verbose: print(f"id: {aid}, finish_date: {aired}, episodes: {ep}")
        time.sleep(1)
    return dict_id_episodes

In [64]:
dict_id_episodes = get_episodes_and_finish_from_list(bad_year_finish.union(bad_episodes))

In [65]:
for k in dict_id_episodes.keys():
    _, end = [normalize_date(parse_secondary(dict_id_episodes[k][1])[i]) for i in range(2)]
    if (not end) or (pd.to_datetime(end).year > 2018):
        end = date.today()
    dict_id_episodes[k] = {
        'episodes': dict_id_episodes[k][0],
        'month_finish': pd.to_datetime(end).month_name(),
        'year_finish': pd.to_datetime(end).year,
        }

df_ep = (
    pd.DataFrame
       .from_dict(dict_id_episodes, orient='index', columns=['episodes', 'month_finish', 'year_finish'])
       .reset_index()
       .rename(columns={'index': 'anime_id'})
)

df_ep

,anime_id,episodes,month_finish,year_finish
0,33922,9.0,November,2025
1,6149,NaN,November,2025
2,34566,293.0,November,2025
3,35590,11.0,November,2025
4,22669,NaN,November,2025
5,8336,NaN,November,2025
6,24977,58.0,November,2025
7,21,NaN,November,2025
8,35478,10.0,March,2018
9,37020,52.0,November,2025


In [66]:
df_joined = df_main.merge(
    df_ep,
    how="left",
    on="anime_id",
    suffixes=("", "_src")
)

for col in ["episodes", "month_finish", "year_finish"]:
    df_joined[col] = df_joined[col].fillna(df_joined[f"{col}_src"])

df_joined = df_joined[[c for c in df_joined.columns if not c.endswith("_src")]]
df_joined[df_joined['anime_id'].isin(bad_year_finish)][["anime_id", "rank", "episodes", "month_finish", "year_finish"]]
df_joined["episodes"] = (pd.to_numeric(df_joined["episodes"], errors="coerce").astype("Int64"))
df_joined["year_start"] = df_joined["year_start"].astype("Int64")
df_joined["year_finish"] = df_joined["year_finish"].astype("Int64")

df_joined[df_joined['anime_id'].isin(bad_year_finish)][["anime_id", "episodes", "month_start", "year_start", "month_finish", "year_finish"]]

,anime_id,episodes,month_start,year_start,month_finish,year_finish
11,21,<NA>,October,1999,November,2025
197,235,<NA>,January,1996,November,2025
745,966,<NA>,April,1992,November,2025
909,1199,<NA>,April,1993,November,2025
1351,1960,<NA>,October,1988,November,2025
1583,2406,<NA>,October,1969,November,2025
2239,4459,<NA>,October,1998,November,2025
2573,6149,<NA>,January,1995,November,2025
2948,8336,<NA>,March,2010,November,2025
3022,8687,<NA>,April,2005,November,2025


We see that we managed to restore almost nothing. Unfortunately the most "big" titles are really have 'Unknown' values in the *'episodes'* variable, because they are still airing. We can not remove these items from the dataframe, because of their large influence, so will consider this result for *'episodes'* column as the final one.
But now **month_finish** and **year_finish** columns are more or less OK.

### SCORE column cleaning

We must prefer *'score_main'* column. To this moment we have only 6 anime titles with zero problem in this column. All of them have rank ~ 10_000. We can remove them.

In [67]:
print(df_joined[df_joined['score_main'] == 0][['rank']])

df_joined = df_joined[df_joined['score_main'] != 0]
df_joined.rename(columns={"score_main": "score"}, inplace=True)
df_joined.drop(columns=["score_secondary"], inplace=True)

         rank
6580   9913.0
6599  10587.0
6600  10118.0
6601  10119.0
6602  10120.0
6603  12719.0


What about duration? In reality all of them really *'Unknown'*. So, we can do nothing. We can hadle with them later.

In [68]:
print("Number of Unknown duration: ", (df_joined['duration'] == 'Unknown').sum())
print("Number of Unknown duration_min: ", (df_joined['duration_min'] == 0).sum())

bad_dur_set = set(df_joined[df_joined['duration'] == 'Unknown']['anime_id'])
bad_dur_min_set = set(df_joined[df_joined['duration_min'] == 0]['anime_id'])

print("Length of intersection: ", len(bad_dur_min_set.intersection(bad_dur_set)))
print("Length of union", len(bad_dur_min_set.union(bad_dur_set)))

df_joined[df_joined['anime_id'].isin(bad_dur_min_set)][['anime_id', 'rank']]

Number of Unknown duration:  15
Number of Unknown duration_min:  37
Length of intersection:  15
Length of union 37


,anime_id,rank
4531,23943,5023.0
4643,25441,7337.0
4953,30232,12356.0
4969,30313,10706.0
5145,31244,8964.0
5161,31361,11424.0
5215,31562,11435.0
5372,32197,10702.0
5389,32275,11410.0
5390,32276,11409.0


After all we can make KS-statistics one more time - between **main** and new, cleaned dataset.

In [69]:
print(len(main_df), len(df_joined))
before_metric = np.array(main_df['score_main'], dtype=float)
after_metric = np.array(df_joined['score'], dtype=float)
print(f"{'Score'}:\t{ks_2samp(before_metric, after_metric)}")
print(f"{'Score'}:\t{np.mean(before_metric)}, {np.mean(after_metric)}")
print(f"{'Score'}:\t{np.std(before_metric)}, {np.std(after_metric)}\n")

for metric in ['members', 'favorites']:
    before_metric = np.array(main_df[f"{metric}_main"], dtype=int) / len(main_df)
    after_metric = np.array(df_joined[metric], dtype=int) / len(df_joined)
    print(f"{metric}:\t{ks_2samp(before_metric, after_metric)}")
    print(f"{metric}:\t{np.mean(before_metric)}, {np.mean(after_metric)}")
    print(f"{metric}:\t{np.std(before_metric)}, {np.std(after_metric)}\n")

6656 6602
Score:	KstestResult(statistic=np.float64(0.0042547931279565635), pvalue=np.float64(0.9999999767841098), statistic_location=np.float64(6.27), statistic_sign=np.int8(1))
Score:	6.849902343749999, 6.863621629809148
Score:	0.9276754800859404, 0.9009127206381894

members:	KstestResult(statistic=np.float64(0.008205646453289214), pvalue=np.float64(0.9770012093408235), statistic_location=np.float64(0.5099158653846154), statistic_sign=np.int8(1))
members:	7.147072470399755, 7.263563403854101
members:	15.803898260326081, 15.985205525947693

favorites:	KstestResult(statistic=np.float64(0.06750685980938177), pvalue=np.float64(1.3937618179339253e-13), statistic_location=np.float64(0.00015024038461538462), statistic_sign=np.int8(1))
favorites:	0.10089695947410086, 0.10255273639917621
favorites:	0.5748385310223462, 0.581832576411304



Columns 'score', 'members' are the same: pvalue is close to 1. But colmn 'favorites' has very small pvalue! Distribution of favorites is so diferent now corresponding to pvalue. But considering the value of *'statistic location'* (the maximum difference in CDF between two distributions).
We see: statistic_location=np.float64(0.00015024038461538462).
As we were making our statistics on the average *'favorites'*, the real region of biggest statistical difference will be:

np.float64(0.00015024038461538462) * len(main_df) = 1.0   < == > region of the anime titles, which was added to *'favorites'* list only by one user.

In [70]:
np.float64(0.00015024038461538462) * len(main_df)

np.float64(1.0)

In [74]:
df_joined.columns

Index(['anime_id', 'image_url', 'score', 'scored_by', 'rank', 'opening_theme',
       'ending_theme', 'duration_min', 'name', 'type', 'studio', 'source',
       'duration', 'genres', 'rating', 'members', 'favorites', 'year_start',
       'year_finish', 'month_start', 'month_finish', 'episodes'],
      dtype='object')

In [ ]:
df_joined.rename(columns={"name": "title"}, inplace=True)
new_order = ['anime_id', 'name', 'rank', 'score', 'type', 'studio', 'genres', 'episodes',
             'month_start', 'year_start', 'month_finish', 'year_finish', 'scored_by', 'rating',
             'members', 'favorites', 'source', 'opening_theme', 'ending_theme', 'image_url',
             'duration', 'duration_min']
df_joined = df_joined[new_order]

In [ ]:
df_joined.to_csv("data/helpers/anime_sterilized.csv")

In [147]:
del main_df
del df_main
del df_joined
gc.collect()

3936

# **2. Users cleaning**

In [ ]:
users_df = pd.read_csv("data/datasets/anime_azathoth42/users_cleaned.csv")
users_df.columns

Index(['username', 'user_id', 'user_watching', 'user_completed', 'user_onhold',
       'user_dropped', 'user_plantowatch', 'user_days_spent_watching',
       'gender', 'location', 'birth_date', 'access_rank', 'join_date',
       'last_online', 'stats_mean_score', 'stats_rewatched', 'stats_episodes'],
      dtype='object')

Drop columns we will not work with

In [ ]:
users_df.drop(
    columns=['user_id', 'user_watching', 'user_completed', 'user_onhold', 'user_dropped',
             'user_plantowatch', 'access_rank', 'last_online', 'stats_rewatched'], inplace=True)
users_df.columns

Implement special location cleaning. The such column is full of bad strings. We write only countries.

In [129]:
print(f"Number of users before cleaning: {users_df['username'].count()}")

Number of users before cleaning: 88475


In [ ]:
cleaner = LocationCleaner()
users_df["country"] = users_df["location"].apply(cleaner.clean)

users_df = cleaner.map_as_unknown(users_df, column="country", min_count=25)

In [ ]:
print(f"Number of users after cleaning: {users_df['username'].count()}")

Clean for value *'Unknown'* in the column **'country'**.

In [110]:
users_df = users_df[users_df['country'] != 'Unknown']

There is one user with clean data but without username. As he is from Serbia, we will call him Dragan.

In [131]:
users_df[users_df['username'].isna()]

,username,user_days_spent_watching,gender,location,birth_date,join_date,stats_mean_score,stats_episodes,country
51311,NaN,39.304167,Male,Serbia,1996-12-30 00:00:00,2010-03-01 00:00:00,6.58,2524,Serbia


In [137]:
users_df.loc[users_df['username'].isna(), 'username'] = 'Dragan'
users_df[users_df['username'] == 'Dragan']

,username,user_days_spent_watching,gender,location,birth_date,join_date,stats_mean_score,stats_episodes,country
51311,Dragan,39.304167,Male,Serbia,1996-12-30 00:00:00,2010-03-01 00:00:00,6.58,2524,Serbia


In [ ]:
users_df.to_csv("data/helpers/users_sterilized.csv")

In [151]:
del users_df
gc.collect()

142

# **3. User-Anime relational table cleaning**

In [ ]:
con = duckdb.connect()

con.execute("""CREATE OR REPLACE TABLE anime AS SELECT * FROM read_csv_auto('data/helpers/anime_sterilized.csv')""")
con.execute("""CREATE OR REPLACE TABLE users AS SELECT * FROM read_csv_auto('data/helpers/users_sterilized.csv')""")
con.execute("""CREATE OR REPLACE TABLE animelist AS SELECT * FROM read_csv_auto('data/anime_azathoth42/animelists_cleaned.csv')""")

t = con.execute("""
            SELECT count(*)
            FROM animelist
            WHERE username IN (SELECT username FROM users)
            AND anime_id IN (SELECT anime_id FROM anime)
            AND my_score != 0""").fetchone()

print(t)

query = """
    SELECT username, anime_id, my_watched_episodes, my_score, my_last_updated
    FROM animelist
    WHERE username IN (SELECT username FROM users)
      AND anime_id IN (SELECT anime_id FROM anime)
      AND my_score != 0
"""
arrow_table = con.execute(query).fetch_arrow_table()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

(15616866,)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
chunk_size = 100_000
num_rows = arrow_table.num_rows
print(num_rows)
start = 0
first = True

with open('data/helpers/animelists_sterilized.csv', 'wb') as f_out:
    pac.write_csv(
        arrow_table.slice(0, 0),   # пустая таблица, только для хедера
        f_out,
        write_options=pac.WriteOptions(include_header=True)
    )

with open('data/helpers/animelists_sterilized.csv', 'ab') as f_out:
    start = 0
    chunk_size = 100_000
    while start < num_rows:
        end = min(start + chunk_size, num_rows)
        chunk = arrow_table.slice(start, end - start)
        
        pac.write_csv(
            chunk,
            f_out,
            write_options=pac.WriteOptions(include_header=False)
        )
        
        print(f"Wrote rows {start} to {end}")
        start = end


15616866
Wrote rows 0 to 100000
Wrote rows 100000 to 200000
Wrote rows 200000 to 300000
Wrote rows 300000 to 400000
Wrote rows 400000 to 500000
Wrote rows 500000 to 600000
Wrote rows 600000 to 700000
Wrote rows 700000 to 800000
Wrote rows 800000 to 900000
Wrote rows 900000 to 1000000
Wrote rows 1000000 to 1100000
Wrote rows 1100000 to 1200000
Wrote rows 1200000 to 1300000
Wrote rows 1300000 to 1400000
Wrote rows 1400000 to 1500000
Wrote rows 1500000 to 1600000
Wrote rows 1600000 to 1700000
Wrote rows 1700000 to 1800000
Wrote rows 1800000 to 1900000
Wrote rows 1900000 to 2000000
Wrote rows 2000000 to 2100000
Wrote rows 2100000 to 2200000
Wrote rows 2200000 to 2300000
Wrote rows 2300000 to 2400000
Wrote rows 2400000 to 2500000
Wrote rows 2500000 to 2600000
Wrote rows 2600000 to 2700000
Wrote rows 2700000 to 2800000
Wrote rows 2800000 to 2900000
Wrote rows 2900000 to 3000000
Wrote rows 3000000 to 3100000
Wrote rows 3100000 to 3200000
Wrote rows 3200000 to 3300000
Wrote rows 3300000 to 340